In [40]:
import yfinance as yf

In [41]:
symbol = "spy"
period = "1d"
interval = "5m"
file_name = f"{symbol}.csv"

In [42]:
ticker = yf.Ticker(symbol)

In [43]:
data = ticker.history(period=period, interval=interval, auto_adjust = True, prepost=True)

In [44]:
data=data.drop(columns=["Dividends", "Stock Splits"])
data['Adj Close']=data['Close']
data.tail()

,Open,High,Low,Close,Volume,Adj Close
Datetime,,,,,,
2021-08-18 14:00:00-04:00,443.320007,444.144012,443.260010,444.130005,1446312,444.130005
2021-08-18 14:05:00-04:00,444.148407,444.190002,443.609985,443.714996,922826,443.714996
2021-08-18 14:10:00-04:00,443.714996,443.855011,442.269989,442.480011,1576309,442.480011
2021-08-18 14:15:00-04:00,442.480011,442.679993,441.700012,442.299988,1920777,442.299988
2021-08-18 14:20:16-04:00,442.299988,442.299988,442.299988,442.299988,0,442.299988


In [45]:
data.to_csv(file_name)
#cleaned up in vscode to change to "Date Time" and to remove "-04:00"

In [ ]:
INITIAL_BUDGET = 5000
BUDGET_USE = 0.50
RISK_PERCENT = 0

In [46]:
import datetime
import math
market_opentime = datetime.time(9,30,0) #6:30 AM PT
arb_tradestoptime = datetime.time(14,0,0) #11 AM PT

In [47]:
market_opentime

datetime.time(9, 30)

In [48]:

from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import macd

In [49]:
# # Load the bar feed from the CSV file
feed = csvfeed.GenericBarFeed(frequency="minute")
feed.addBarsFromCSV(symbol, file_name)

In [50]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [51]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__position = None
        #Looking for a MACD crossover
        #self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)
        self.__macd = macd.MACD(feed[instrument].getCloseDataSeries(), 12, 26, 9)
        self.__instrument = instrument

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info(f"BUY at ${execInfo.getPrice()}")
    
    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info(f"SELL at ${execInfo.getPrice()}")
        self.__position = None
    
    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__macd.getHistogram()[-1] is None:
            return
        if self.__macd.getHistogram()[-2] is None:
            return
        
        bar = bars[self.__instrument]
        if self.__position is None:
            if market_opentime <= bar.getDateTime().time() <= arb_tradestoptime:
                if self.__macd.getHistogram()[-1]>0 and self.__macd.getHistogram()[-2]<0:
                    print(f"Crossover UP detected at {bar.getDateTime()}. {symbol.upper()} price was ${safe_round(bar.getClose(), 2)}. Buy some {math.ceil(bar.getClose())} CALLS.")
                    self.__position = self.enterLong(self.__instrument, 10, True)                   
                    
                if self.__macd.getHistogram()[-1]<0 and self.__macd.getHistogram()[-2]>0:
                    self.__position = self.enterShort(self.__instrument, 10, True)                    
                    print(f"Crossover DOWN detected at {bar.getDateTime()}. {symbol.upper()} price was ${safe_round(bar.getClose(), 2)}. Buy some {math.floor(bar.getClose())} PUTS.") 

        if self.__position:
            if self.__position.getEntryOrder().getAction() == 4:
                if self.__macd.getHistogram()[-1] > self.__macd.getHistogram()[-2]:
                    print(f"Momentum slowing at {bar.getDateTime()}.")
                    self.__position.exitMarket()
            else:
                if self.__macd.getHistogram()[-1] < self.__macd.getHistogram()[-2]:
                    print(f"Momentum slowing at {bar.getDateTime()}.")
                    self.__position.exitMarket()
                    
        
            
                
            

        #self.info("%s %s" % (bar.getClose(), safe_round(self.__macd.getHistogram()[-1], 4)))

In [52]:
ms1 = MyStrategy(feed, symbol)
ms1.run()

2021-08-18 09:50:00 strategy [INFO] BUY at $443.3399963378906
2021-08-18 10:30:00 strategy [INFO] SELL at $444.19000244140625
2021-08-18 10:55:00 strategy [INFO] BUY at $444.0299987792969
2021-08-18 11:25:00 strategy [INFO] SELL at $443.5
2021-08-18 12:20:00 strategy [INFO] BUY at $443.5469970703125
2021-08-18 12:25:00 strategy [INFO] SELL at $443.20001220703125
2021-08-18 13:00:00 strategy [INFO] BUY at $443.3800048828125
2021-08-18 13:20:00 strategy [INFO] SELL at $443.1700134277344
2021-08-18 13:30:00 strategy [INFO] BUY at $443.0450134277344
2021-08-18 13:35:00 strategy [INFO] SELL at $443.125
2021-08-18 13:40:00 strategy [INFO] BUY at $443.1850891113281
2021-08-18 14:15:00 strategy [INFO] SELL at $442.4800109863281


Crossover UP detected at 2021-08-18 09:45:00. SPY price was $443.34. Buy some 444 CALLS.
Momentum slowing at 2021-08-18 10:25:00.
Crossover DOWN detected at 2021-08-18 10:50:00. SPY price was $444.02. Buy some 444 PUTS.
Momentum slowing at 2021-08-18 11:20:00.
Crossover UP detected at 2021-08-18 12:15:00. SPY price was $443.55. Buy some 444 CALLS.
Momentum slowing at 2021-08-18 12:20:00.
Crossover UP detected at 2021-08-18 12:55:00. SPY price was $443.37. Buy some 444 CALLS.
Momentum slowing at 2021-08-18 13:15:00.
Crossover DOWN detected at 2021-08-18 13:25:00. SPY price was $443.05. Buy some 443 PUTS.
Momentum slowing at 2021-08-18 13:30:00.
Crossover UP detected at 2021-08-18 13:35:00. SPY price was $443.19. Buy some 444 CALLS.
Momentum slowing at 2021-08-18 14:10:00.


In [ ]:
#reverse order:
option time price 
444P @10:01 $0.76    $0.81@10:03
444C @07:26 $0.64    $0.68@07:30
443C @09:21 $0.36    $0.82@12:27
443P @07:01 $0.24    $0.37@07:36
442P @9:40 $0.65     $0.70@09:47
443C @8:35 $0.85     $1.02@08:48
442P @7:35 $0.82     $1.24@07:54
443C @11:00 $
    
445P 6:35 0.40-0.47
446C 6:45 0.28-0.44
445P 7:00 0.30-0.55calc
446C 7:20 0.28-0.36
445P 7:25 0.24-0.34
446C 7:33 0.24
    
    8/17 445C 1.00 OPEn @ 7:25 1.20 High @ 7:29 1.13 Open @ 7:30
    8/17 443P 0.89 Open @ 8:25 1.11 High @ 8:29 1.07 Open @ 8:30
    8/17 443C 1.04 Open @ 10:25 1.25 High @ 10:29 1.17 Open @ 10:35
    
    8/18 444C @6:45 0.54O 1.13H @7:19 1.05O @ 7:25
    8/18 444P @7:50 0.77O 1.16H@8:18 0.99O@8:20   
    8/18 444C @9:15 0.44O  0.46H @9:15 0.42O@9:20
    8/18 444C @9:55 0.28O 0.34H@10:00 0.29O@10:15
    8/18 443P @10:25 0.57O 0.65H@10:27 0.56O@10:30
    8/18 444C @10:35 0.24O  0.62H@11:05 0.38O@11:10
        
    